# Linear Regression with Gradient Descent

In this notebook we will use Turi Create along with numpy to solve for the regression weights with gradient descent.

We will:
* Convert an SFrame into a Numpy array
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size and tolerance.
* Use the gradient descent function to estimate regression weights for multiple features

Make sure you have the latest version of Turi Create and import the necessary libraries

In [71]:
import turicreate
import pandas as pd
import numpy as np
from math import sqrt

# Load in House Sales Data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [62]:
sales = turicreate.SFrame(r'\home_data.sframe')
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930.0,1.0,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819.0,2.0,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711.0,1.0,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470.0,1.0,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560.0,2.0,0


# Convert to Numpy Array

Although SFrames offer a number of benefits to users (especially when using Big Data and built-in Turi Create functions), in order to understand the details of the implementation of algorithms it's important to work with a library that allows for direct (and optimized) matrix operations. Numpy is a Python solution to work with matrices (or any multi-dimensional "array").

Recall that the predicted value given the weights and the features is just the dot product between the feature and weight vector. Similarly, if we put all of the features row-by-row in a matrix then the predicted value for *all* the observations can be computed by right multiplying the "feature matrix" by the "weight vector". 
 
Now we will write a function that will accept an SFrame, a list of feature names (['sqft_living', 'sqft_living_15']) and a target feature ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an intercept)
* A numpy array containing the values of the output


In [79]:
def get_numpy_data(data_frame, features, output):
    data_frame['constant'] = 1 # this is how you add a constant column to an frame
    
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    
    # select the columns of data_frame given by the features list into the frame features_frame (now including constant):
    features_frame=data_frame[features]
    
    # the following line will convert the features_frame into a numpy matrix:
    feature_matrix = features_frame.to_numpy()
    
    # assign the column of data_frame associated with the output to the SArray output_sarray
    output_sarray=data_frame[output]

    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

We now define two additional functions:

* One to compute the predictions for an entire matrix of features given the matrix and the weights:
* One to compute the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.


In [80]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    predictions=np.dot(feature_matrix, weights)

    return(predictions)

def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    derivative=2*(np.dot(errors, feature))
    
    return(derivative)

# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple: given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the step siz'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

In [81]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        # compute the predictions based on feature_matrix and weights using the predict_output() function
        predictions=predict_output(feature_matrix, weights)
        
        # compute the errors as predictions - output
        errors=predictions-output

        gradient_sum_squares = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix[:,i])
            
            # add the squared value of the derivative to the gradient sum of squares (for assessing convergence)
            gradient_sum_squares=gradient_sum_squares+(derivative**2)
            
            
            # subtract the step size times the derivative from the current weight
            weights[i]=weights[i]-step_size*derivative
            
        # compute the square-root of the gradient sum of squares to get the gradient magnitude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

A few things to note before we run the gradient descent. Since the gradient is a sum over all the data points and involves a product of an error and a feature the gradient itself will be very large since the features are large (squarefeet) and the output is large (prices). So while you might expect "tolerance" to be small, small is only relative to the size of the features. 

For similar reasons the step size will be much smaller than you might expect but this is because the gradient has such large values.

In [76]:
# let's test out the gradient descent
simple_features=['sqft_living', 'sqft_living15']  # sqft_living15 is the average squarefeet for the nearest 15 neighbors.
my_output='price'
(feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [77]:
my_weights=regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)
my_weights

array([-9.99999615e+04,  2.42150173e+02,  6.86243532e+01])

Let's compare the coefficients we get from our algorithm to those which are obtained from Turi Create's linear regression command 

In [82]:
test_model = turicreate.linear_regression.create(sales, target = 'price', features = ['sqft_living', 'sqft_living15'], 
                                                    validation_set = None, verbose=False)
test_model.coefficients

name,index,value,stderr
(intercept),None,-98862.4128972,5419.30151481
sqft_living,None,242.214377072,2.94031519674
sqft_living15,None,68.0419653568,3.94009336002


As you can see, the coefficient weights on sqft_living and sqft_living15 estimated using Turi Create mirror the weights estimated by our algorithm!